In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageNet, CIFAR10

from PyTorch_CIFAR10.cifar10_models.mobilenetv2 import mobilenet_v2
from PyTorch_CIFAR10.UTILS_TORCH import *

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Currently using: '{device}'")

Currently using: 'cuda'


In [3]:
# Hyperparameters
NUM_EPOCHS = 100
BATCH_SIZE = 64
LEARN_RATE = 0.001
ALPHA = 0.1
TEMP = 3
NUM_WORKERS = 16
OVERLAY_PROB = 0.25

In [4]:
IGS = "PyTorch_CIFAR10/Captum_IGs.npy"
print(f"Using IGs: {IGS}")

# Load the precomputed IGs
igs = np.load(IGS)
print(f"IGs shape: {igs.shape}")

Using IGs: PyTorch_CIFAR10/Captum_IGs.npy
IGs shape: (50000, 32, 32)


In [5]:
precomputed_logits = np.load("PyTorch_CIFAR10/data/cifar10_logits.npy")
print("Shape of teacher_logits:", precomputed_logits.shape)

Shape of teacher_logits: (50000, 10)


In [6]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

student_aug = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

In [7]:
# Define the full training dataset without augmentation for splitting
train_dataset = CIFAR10_KD(
    root="PyTorch_CIFAR10/data",
    train=True,
    transform=student_aug,
    precomputed_logits=precomputed_logits,
)

# Load the data into batches
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=True,
)

# load student data
test_data = CIFAR10(
    root="PyTorch_CIFAR10/data",
    train=False,
    transform=transform,
)

test_loader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    persistent_workers=True,
)

Files already downloaded and verified


In [8]:
Teacher = mobilenet_v2(pretrained=True)
Teacher.to(device);

In [9]:
teacher_params = count_parameters(Teacher)

In [10]:
Student = SmallerMobileNet(mobilenet_v2(pretrained=False))
Student.to(device);

In [11]:
student_params = count_parameters(Student)

In [12]:
teacher_params / student_params

4.115345410654685

# First Student

In [13]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-3])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [14]:
Student_1 = Smaller(mobilenet_v2(pretrained=False))
Student_1.to(device);

In [15]:
S1 = count_parameters(Student_1)
teacher_params / S1

2.194111842040726

In [16]:
Student_1, acc_1 = train_eval_kd(
    student=Student_1,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_2.19_kd.csv",
)

Epoch 1/100: Loss: 1.7892,            Accuracy: 44.54%


Epoch 2/100: Loss: 1.4349,            Accuracy: 61.77%


Epoch 3/100: Loss: 1.2586,            Accuracy: 70.02%


Epoch 4/100: Loss: 1.1499,            Accuracy: 75.13%


Epoch 5/100: Loss: 1.0790,            Accuracy: 78.09%


...

Epoch 96/100: Loss: 0.5817,            Accuracy: 98.47%


Epoch 97/100: Loss: 0.5785,            Accuracy: 98.59%


Epoch 98/100: Loss: 0.5789,            Accuracy: 98.64%


Epoch 99/100: Loss: 0.5785,            Accuracy: 98.57%


Epoch 100/100: Loss: 0.5787,            Accuracy: 98.60%


In [17]:
print(f"Test Acc = {acc_1:.2f}%")

Test Acc = 91.59%


# Second Student

In [18]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-7])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [19]:
Student_2 = Smaller(mobilenet_v2(pretrained=False))
Student_2.to(device);

In [20]:
S2 = count_parameters(Student_2)
teacher_params / S2

7.286700938902897

In [21]:
Student_2, acc_2 = train_eval_kd(
    student=Student_2,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_7.29_kd.csv",
)

Epoch 1/100: Loss: 1.6530,            Accuracy: 51.03%


Epoch 2/100: Loss: 1.3368,            Accuracy: 66.53%


Epoch 3/100: Loss: 1.1844,            Accuracy: 73.89%


Epoch 4/100: Loss: 1.1028,            Accuracy: 77.78%


Epoch 5/100: Loss: 1.0465,            Accuracy: 80.12%


...

Epoch 96/100: Loss: 0.5921,            Accuracy: 98.47%


Epoch 97/100: Loss: 0.5899,            Accuracy: 98.55%


Epoch 98/100: Loss: 0.5903,            Accuracy: 98.56%


Epoch 99/100: Loss: 0.5903,            Accuracy: 98.52%


Epoch 100/100: Loss: 0.5901,            Accuracy: 98.54%


In [22]:
print(f"Test Acc = {acc_2:.2f}%")

Test Acc = 90.88%


# Third Student

In [23]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-9])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [24]:
Student_3 = Smaller(mobilenet_v2(pretrained=False))
Student_3.to(device);

In [25]:
S3 = count_parameters(Student_3)
teacher_params / S3

12.042134619733172

In [26]:
Student_3, acc_3 = train_eval_kd(
    student=Student_3,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_12.04_kd.csv",
)

Epoch 1/100: Loss: 1.6878,            Accuracy: 49.69%


Epoch 2/100: Loss: 1.3657,            Accuracy: 66.01%


Epoch 3/100: Loss: 1.2234,            Accuracy: 72.99%


Epoch 4/100: Loss: 1.1455,            Accuracy: 76.40%


Epoch 5/100: Loss: 1.0914,            Accuracy: 79.00%


...

Epoch 96/100: Loss: 0.6485,            Accuracy: 97.78%


Epoch 97/100: Loss: 0.6478,            Accuracy: 97.80%


Epoch 98/100: Loss: 0.6464,            Accuracy: 97.90%


Epoch 99/100: Loss: 0.6463,            Accuracy: 97.83%


Epoch 100/100: Loss: 0.6442,            Accuracy: 97.87%


In [27]:
print(f"Test Acc = {acc_3:.2f}%")

Test Acc = 90.52%


# Fourth Student

In [28]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-11])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [29]:
Student_4 = Smaller(mobilenet_v2(pretrained=False))
Student_4.to(device);

In [30]:
S4 = count_parameters(Student_4)
teacher_params / S4

28.974816695598104

In [31]:
Student_4, acc_4 = train_eval_kd(
    student=Student_4,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_28.97_kd.csv",
)

Epoch 1/100: Loss: 1.7102,            Accuracy: 48.38%


Epoch 2/100: Loss: 1.4236,            Accuracy: 63.20%


Epoch 3/100: Loss: 1.3136,            Accuracy: 68.54%


Epoch 4/100: Loss: 1.2503,            Accuracy: 71.94%


Epoch 5/100: Loss: 1.1971,            Accuracy: 74.37%


...

Epoch 96/100: Loss: 0.7817,            Accuracy: 94.00%


Epoch 97/100: Loss: 0.7788,            Accuracy: 94.14%


Epoch 98/100: Loss: 0.7787,            Accuracy: 94.18%


Epoch 99/100: Loss: 0.7775,            Accuracy: 94.23%


Epoch 100/100: Loss: 0.7758,            Accuracy: 94.29%


In [32]:
print(f"Test Acc = {acc_4:.2f}%")

Test Acc = 88.07%


# Fifth Student

In [33]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-13])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [34]:
Student_5 = Smaller(mobilenet_v2(pretrained=False))
Student_5.to(device);

S5 = count_parameters(Student_5)
teacher_params / S5

54.5931657310227

In [35]:
Student_5, acc_5 = train_eval_kd(
    student=Student_5,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_54.59_kd.csv",
)

Epoch 1/100: Loss: 1.7350,            Accuracy: 47.72%


Epoch 2/100: Loss: 1.4954,            Accuracy: 59.82%


Epoch 3/100: Loss: 1.3980,            Accuracy: 64.59%


Epoch 4/100: Loss: 1.3371,            Accuracy: 67.66%


Epoch 5/100: Loss: 1.2927,            Accuracy: 70.06%


...

Epoch 96/100: Loss: 0.8913,            Accuracy: 89.86%


Epoch 97/100: Loss: 0.8889,            Accuracy: 90.05%


Epoch 98/100: Loss: 0.8884,            Accuracy: 90.02%


Epoch 99/100: Loss: 0.8873,            Accuracy: 89.99%


Epoch 100/100: Loss: 0.8857,            Accuracy: 90.20%


In [36]:
print(f"Test Acc = {acc_5:.2f}%")

Test Acc = 84.80%


# Sixth Student

In [37]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-15])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [38]:
Student_6 = Smaller(mobilenet_v2(pretrained=False))
Student_6.to(device);

S6 = count_parameters(Student_6)
teacher_params / S6

139.426630095998

In [39]:
Student_6, acc_6 = train_eval_kd(
    student=Student_6,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_139.43_kd.csv",
)

Epoch 1/100: Loss: 1.8369,            Accuracy: 42.78%


Epoch 2/100: Loss: 1.6156,            Accuracy: 54.33%


Epoch 3/100: Loss: 1.5241,            Accuracy: 58.99%


Epoch 4/100: Loss: 1.4615,            Accuracy: 62.12%


Epoch 5/100: Loss: 1.4227,            Accuracy: 63.93%


...

Epoch 96/100: Loss: 1.0819,            Accuracy: 81.06%


Epoch 97/100: Loss: 1.0803,            Accuracy: 81.07%


Epoch 98/100: Loss: 1.0800,            Accuracy: 81.02%


Epoch 99/100: Loss: 1.0825,            Accuracy: 81.13%


Epoch 100/100: Loss: 1.0777,            Accuracy: 81.20%


In [40]:
print(f"Test Acc = {acc_6:.2f}%")

Test Acc = 78.73%


# Seventh Student

In [41]:
class Smaller(nn.Module):
    def __init__(self, original_model):
        super(Smaller, self).__init__()
        self.features = nn.Sequential(*list(original_model.features.children())[:-17])

        # Dynamically find the output channels of the last convolutional block used
        # Iterate backwards over the blocks to find the last convolutional layer
        for block in reversed(self.features):
            if hasattr(block, "conv"):
                # If the block contains a 'conv' attribute, likely to be a sequential module
                if hasattr(block.conv, "__iter__"):
                    # Find the last Conv2d module in the block
                    for layer in reversed(block.conv):
                        if isinstance(layer, nn.Conv2d):
                            num_output_channels = layer.out_channels
                            break
                    break
            elif isinstance(block, nn.Conv2d):
                num_output_channels = block.out_channels
                break

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_output_channels, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = self.classifier(x)
        return x

In [42]:
Student_7 = Smaller(mobilenet_v2(pretrained=True))
Student_7.to(device);

S7 = count_parameters(Student_7)
teacher_params / S7

1121.7061183550652

In [43]:
Student_7, acc_7 = train_eval_kd(
    student=Student_7,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_1121.71_kd.csv",
)

Epoch 1/100: Loss: 2.1050,            Accuracy: 27.42%


Epoch 2/100: Loss: 1.9149,            Accuracy: 36.97%


Epoch 3/100: Loss: 1.8450,            Accuracy: 41.60%


Epoch 4/100: Loss: 1.8054,            Accuracy: 44.44%


Epoch 5/100: Loss: 1.7825,            Accuracy: 45.79%


...

Epoch 96/100: Loss: 1.6128,            Accuracy: 55.17%


Epoch 97/100: Loss: 1.6132,            Accuracy: 55.23%


Epoch 98/100: Loss: 1.6091,            Accuracy: 55.18%


Epoch 99/100: Loss: 1.6093,            Accuracy: 55.07%


Epoch 100/100: Loss: 1.6127,            Accuracy: 54.91%


In [44]:
print(f"Test Acc = {acc_7:.2f}%")

Test Acc = 50.36%


# Normal Student

In [45]:
Student_8 = SmallerMobileNet(mobilenet_v2(pretrained=False))
Student_8.to(device)

S8 = count_parameters(Student_8)
teacher_params / S8

4.115345410654685

In [46]:
Student_8, acc_8 = train_eval_kd(
    student=Student_8,
    train_loader=train_loader,
    test_loader=test_loader,
    epochs=NUM_EPOCHS,
    lr=LEARN_RATE,
    TEMP=TEMP,
    ALPHA=ALPHA,
    device=device,
    csv_path="PyTorch_CIFAR10/Histories/Compression/Compression_of_4.12_kd.csv",
)

Epoch 1/100: Loss: 1.7147,            Accuracy: 47.50%


Epoch 2/100: Loss: 1.3525,            Accuracy: 66.02%


Epoch 3/100: Loss: 1.1863,            Accuracy: 73.73%


Epoch 4/100: Loss: 1.0920,            Accuracy: 77.75%


Epoch 5/100: Loss: 1.0346,            Accuracy: 80.26%


...

Epoch 96/100: Loss: 0.5781,            Accuracy: 98.66%


Epoch 97/100: Loss: 0.5791,            Accuracy: 98.64%


Epoch 98/100: Loss: 0.5755,            Accuracy: 98.77%


Epoch 99/100: Loss: 0.5751,            Accuracy: 98.81%


Epoch 100/100: Loss: 0.5768,            Accuracy: 98.71%


In [47]:
print(f"Test Acc = {acc_8:.2f}%")

Test Acc = 92.08%
